Remarque : fait à l'arrache dans le notebook pour l'instant, déplacement dans un .py plus tard

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math

In [2]:
#entrée : positional encoding + embeddings
#ajouter layer norm

In [ ]:
# Paramètres

nb_encoders = 6
nb_heads = 8
vector_size = 512
vocab_size = 1000
ffn_hidden_size = 100


In [4]:
class Transformer(nn.Module):
    def __init__(self, decoder):
        super(Transformer, self).__init__()
        self.decoder = decoder

    def forward(self, x):
        for x in range(5):
            x = self.decoder(x)
        Traitements_posts(x)
        return x

In [ ]:
class Decoder(nn.Module):
    "Decoder block applying multihead attention mechanism folowed by a feedforward network"
    def __init__(self, multihead_attention, feedforward_network):
        super(Decoder, self).__init__()
        self.multihead_attention = multihead_attention
        self.feedforward_network = feedforward_network
        self.layernorm = nn.modules.normalization.LayerNorm

    def forward(self, x):
        mha = self.multihead_attention(self.layernorm(x))
        x += mha
        ffo = self.feedforward_network(self.layernorm(x))
        x += ffo
        return x
    

In [ ]:
class FeedforwardNetwork(nn.Module):
    def __init__(self, vector_size, hidden_size):
        "Classic Feedforward Network with two layers"
        super(FeedforwardNetwork, self).__init__()
        self.hidden_size  = hidden_size
        self.fc1 = nn.Linear(vector_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, vector_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class MultiHeadAttention(nn.Module):
    "MultiHead Attention Block"
    def __init__(self, nb_heads, features_size, vector_size):
        super(SelfAttention, self).__init__()
        self.nb_heads = nb_heads
        self.features_size = features_size
        #A changer
        self.w_q = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_k = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_v = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_0 = nn.Linear(self.nb_heads*vector_size, vector_size)
        
    def mask_w(w):
        pass
    
    def attention(self, q, k, v):
        w = torch.matmul(q, k.transpose(1,0)) / math.sqrt() #v.shape[-1].value
        w = mask_w(w)
        w = F.softmax(w)
        a = tf.matmul(w, v)
        return a
        
    def forward(self):
        q = self.w_q(x).view(self.nb_heads, -1)
        k = self.w_k(x).view(self.nb_heads, -1)
        v = self.w_v(x).view(self.nb_heads, -1)
        a = attention(q, k, v) #redimensionner ?
        a = self.w_0(a)
        return a
